# Elemental abundance distribution and comparison with observation
This notebook allows to plot the elemental abundance distribution and compare it with observations. Elemental abundance distributions can be scaled in different ways and this notebook shows all options we use for research. 

ppn run results are saved run directory created with the `save_run.sh` script. Runs are of one of three types as selected in the `ppn_frame.input` file. The cases are
1. `2ndpeak`
2. `Sakurai`
3. `weak`

In [ ]:
# select a case from the following ones:
# case = '2ndpeak'
# case = 'Sakurai'
case = 'weak'

### Initializing the notebook

In [ ]:
%pylab ipympl
from nugridpy import ppn
from nugridpy import utils

The following cell identifies default analysis parameters for each run, such as the time steps to be compared to observations.

In [ ]:
if case == 'Sakurai':
    ppn_cycle = 509 # most plots are made for this cycle    
    ppn_dilute = 0.8 # adjust dilution coefficient for the last plot
    star=case # observed elemental abundances are supposed to be in file star.txt
    Zpin = 39 # atomic number of the element used to pin abundance distribution profile
    selected_cycles = [495,500,505,509] # observed abundances are fitted for these cycles
    clr = utils.colourblind(0)
elif case == '2ndpeak':
    ppn_cycle = 830 # most plots are made for this cycle    
    ppn_dilute = 0.0025 # adjust dilution coefficient for the last plot
    star=case # observed elemental abundances are supposed to be in file star.txt
    Zpin = 57 # atomic number of the element used to pin abundance distribution profile
    selected_cycles = [800,830,860,891] # observed abundances are fitted for these cycles
    clr = utils.colourblind(1)
elif case == 'weak':
    ppn_cycle = 511 # most plots are made for this cycle    
    ppn_dilute = 0.0025 # adjust dilution coefficient for the last plot
    star=case # observed elemental abundances are supposed to be in file star.txt
    Zpin = 33 # atomic number of the element used to pin abundance distribution profile
    selected_cycles = [509,510,511,512] # observed abundances are fitted for these cycles
    clr = utils.colourblind(2)

In [ ]:
rundir = 'master-result'
#rundir = 'I135ng_up3'

In [ ]:
# read in ppn run results
pa=ppn.abu_vector('../'+rundir)

## Basic elemental abundance plot
The `elemental_abund` method of an `abu_vector` method plots the decayed elemental mass fraction abundances for a given cycle.

Use command
```Python
pa.elemental_abund?
```
to view the doc string with all options explained.

Below we first show  the most basic plot, and then show variations, including comparison with observed abundances and $\chi^2$ fitting.

In [ ]:
pa.elemental_abund?

In [ ]:
close(1);figure(1)

pa.elemental_abund(0,ylim=[-14,-2],zrange=[25,85], label='cycle  0')
pa.elemental_abund(ppn_cycle,ylim=[-14,-2],zrange=[25,85], label='cycle '+str(ppn_cycle), colour=clr, \
                   plotlines='--', plotlabels=True, mark='x')
legend(loc=2)

## Relative mass fractions

Often we want to know how the abundance distribution has changed with respect to an earlier time step or a model from a different run. We can plot mass fractions with reference to another dataset using the ref option. By setting ref > -1 we divide the mass fraction for the selected cycle by the mass fraction for cycle `ref=N`. The most common use is to compare to the initial abundances stored in cycle `0` (`ref=0`). The y-label will show the reference cycle.

In [ ]:
close(2);figure(2)

pa.elemental_abund(ppn_cycle,ref=0,\
                   zrange=[25,85],ylim=[-3,8],colour=clr)

#### Reference files 

We can also use a given abundance file as a reference in the USEEPP data format, such as 'iniab1.4E-02As09.ppn', by using the ref_filename option.
This will set ref=-2, however setting the ref value manually will override the solar option.

In [ ]:
pwd

In [ ]:
close(3);figure(3)

# initial (usually, solar) abundances are supposed to be in this directory
iniab_dir = '../../USEEPP/'

pa.elemental_abund(ppn_cycle,ref_filename=iniab_dir+'iniab1.4E-02As09.ppn',\
                   zrange=[25,85],ylim=[-5,5],colour=clr)
ylabel('$\log_{10}\,X/X_\odot$')

## Scaling by pinning to an observed elemental abundance

To improve comparison to observed data the elemental abundanc distribution can be _pinned_ to a certain value. All abundances are scaled by the same factor so that the simulated abundance of a particular element (the pinning element) matches exactly a particular value at the plotted cycle. For example, the star HD 94028 as a value of [As/Fe] of 0.63 and by selecting As as the pinning element and setting the pinning value to 0.63 the simul;ated abundance distribution is scaled for the given cycle such that the simulated and observed As abundance are the same in the plot. This method makes sense if the observed abundances are believed to originate in just one site and are dominantly present with large overabundances. 

The pinning process calculates an 'offset' which is the difference between the cycle [X/Fe] value and the observed [X/Fe] value.

    offset = [X/Fe]_obs - [X/Fe]_cycle
    
The offset is then added (in log space) to every element to shift the plot.

### Using the z_pin option
First we just do one cycle `ppn_cycle` and the pinning element (`z_pin` option) specified in the parameter block for the given case at the beginning of this notebook.

Prepare a list of atomic numbers of the elements for which a $\chi^2$ test will be performed:

In [ ]:
# list of Z for test
if case == 'Sakurai':
    ztest = [37,38,39,40]
elif case == '2ndpeak':
    ztest = []
    zbeg = 56; zend = 72
    for z in range(zbeg,zend+1):
        ztest.append(z)
elif case == 'weak':
    ztest = [32,34,38,39,40,42,44,46,48]
    
print (case,type(ztest),ztest)

#### Using an observation data file

We can use an observed abundance distribution to set the pinning value. The file name is specified with the pin_filename option. The requirements for the file are columns containing `Z`, `[X/Fe]`, `sigma_xfe`, and `ul` (charge number, [X/Fe], uncertainty in the [X/Fe] value, and an upper limit binary).

The observation is compared to the value of [X/Fe] for the simulation number and the reference cycle. A $\chi^2$ test is conducted for elements with Z in the range provided with the `zchi2` argument and result is returned from the function call. The test takes a few seconds for each cycle.

In [ ]:
close(10);figure(10)

if case == 'Sakurai':
    chi2 = pa.elemental_abund(ppn_cycle,ref=0,z_pin=Zpin,label=case,\
                pin_filename='obsab_'+case+'.dat',zchi2=ztest,\
                ylim=[1,4],zrange=[36.5,41],colour=clr,plotlabels=True)
elif case == '2ndpeak':
    chi2 = pa.elemental_abund(ppn_cycle,ref=0,z_pin=Zpin,label=case,\
                pin_filename='obsab_'+case+'.dat',zchi2=ztest,\
                ylim=[0.5,3.5],zrange=[54.5,82],colour=clr,plotlabels=True)
elif case == 'weak':
    chi2 = pa.elemental_abund(ppn_cycle,ref=0,z_pin=Zpin,label=case,\
                pin_filename='obsab_'+case+'.dat',zchi2=ztest,\
                ylim=[-1,2],zrange=[27.5,50],colour=clr,plotlabels=True)    
legend(loc=1,frameon=False)
print("Chi-squared: {:.3f}".format(chi2[0]))

#### Performing $\chi^2$ tests for a series of cycles to determine best fit
Select the cycles for which the test will be performed.


In [ ]:
cycles = selected_cycles  # use the default list defined in the parameter list at the top
ncyc = len(cycles)
chi2 = linspace(0,0,ncyc) # this array holds the chi2 values, one for each cycle
print (cycles)

In [ ]:
close(4);figure(4)

for i in range(ncyc):
    ifplotlabels = False
    if i == 0:
        ifplotlabels = True
    icyc = cycles[i]
    if case == 'Sakurai':
        chi2[i] = pa.elemental_abund(icyc,ref=0,z_pin=Zpin,\
                   pin_filename='obsab_'+case+'.dat',zchi2=ztest,\
                   ylim=[1.0,4.0],zrange=[36.5,41],colour=utils.colourblind(i),plotlabels=ifplotlabels,label='timestep '+str(icyc))
    elif case == '2ndpeak':
        chi2[i] = pa.elemental_abund(icyc,ref=0,z_pin=Zpin,\
                   pin_filename='obsab_'+case+'.dat',zchi2=ztest,\
                   ylim=[0.5,3.5],zrange=[54.5,82],colour=utils.colourblind(i),plotlabels=ifplotlabels,label='timestep '+str(icyc))
    elif case == 'weak':
        chi2[i] = pa.elemental_abund(icyc,ref=0,z_pin=Zpin,\
                   pin_filename='obsab_'+case+'.dat',zchi2=ztest,\
                   ylim=[-1,2],zrange=[27.5,50],colour=utils.colourblind(i),plotlabels=ifplotlabels,label='timestep '+str(icyc))
legend(frameon=False)
grid(False)

ylabel('[X/Fe]')

Next we print the $\chi^2$ values. From these cycles number 830 has the smallest value and represents the best fit.

In [ ]:
for i in range(ncyc):
    print (i,cycles[i],chi2[i])

## Other plotting options


### Using a manual pin
Without an observational data file a pin value can be manually entered.

In [ ]:
close(5);figure(5)
if case == 'Sakurai':
    pa.elemental_abund(ppn_cycle,ref=0,z_pin=Zpin,pin=2.84,\
                   ylim=[1,4],zrange=[36.5,41],colour=clr)
elif case == '2ndpeak':
    pa.elemental_abund(ppn_cycle,ref=0,z_pin=Zpin,pin=2.10,\
                   ylim=[0.5,3.5],zrange=[54.5,82],colour=clr)
elif case == 'weak':
    pa.elemental_abund(ppn_cycle,ref=0,z_pin=Zpin,pin=0.62,\
                   ylim=[-1,2],zrange=[27.5,50],colour=clr)

### log eps
Sometimes observations are given in the unit $\log \epsilon$ and predicted abundances can be given in those units by setting `logeps=True`.

In [ ]:
close(6);figure(6)
pa.elemental_abund(ppn_cycle,logeps=True,zrange=[1,85],\
                   ylim=[-6,10],colour=clr)

### Dilution and direct [X/Fe] calculations
The above method of calculating the [X/Fe] value using a pinning mechanism is an approximation, using log(X/X_sol) as a proxy for [X/Fe]. This approximation is made for stars whose internal convective behaviour cannot be or has not been modeled. 

We can directly calculate [X/Fe] for a star using the dilution option to define the dilution coefficient for our case. This method has inherent limitations in that for many cases, [X/Fe]<0 will not be feasible as local Fe is depleted during nucleosynthesis processes.

In [ ]:
close(7);figure(7)
obsab_dir = './'
if case == 'Sakurai':
    pa.elemental_abund(ppn_cycle,ref_filename=iniab_dir+'iniab1.4E-02As09.ppn',dilution=ppn_dilute,\
                pin_filename=obsab_dir+'obsab_'+case+'.dat',\
                ylim=[1.0,4.0],zrange=[36.5,41],colour=clr,plotlines='-',plotlabels=True,\
                label=case+', cycle '+str(ppn_cycle),mark='')
elif case == '2ndpeak':
    pa.elemental_abund(ppn_cycle,ref_filename=iniab_dir+'iniab1.4E-02As09.ppn',dilution=0.0025,\
                pin_filename=obsab_dir+'obsab_'+case+'.dat',\
                ylim=[0.5,3.5],zrange=[54.5,82],colour=clr,plotlines='-',plotlabels=True,\
                label=case+', cycle '+str(ppn_cycle),mark='') 
elif case == 'weak':
    pa.elemental_abund(ppn_cycle,ref_filename=iniab_dir+'iniab1.4E-02As09.ppn',dilution=0.00065,\
                pin_filename=obsab_dir+'obsab_'+case+'.dat',\
                ylim=[-1,2],zrange=[27.5,50],colour=clr,plotlines='-',plotlabels=True,\
                label=case+', cycle '+str(ppn_cycle),mark='') 
    
ylabel('[X/Fe]')
grid(False)
legend(frameon=False)

## Isotopic abundance plots

In [ ]:
decayed = False
close(8);figure(8)
obsab_dir = './'
if case == 'Sakurai':
    pa.iso_abund(ppn_cycle,decayed=decayed,amass_range=(70,110),ylim=[-9,-5])
elif case == '2ndpeak':
    pa.iso_abund(ppn_cycle,decayed=decayed,amass_range=(120,170),ylim=[-9,-5])
elif case == 'weak':
    pa.iso_abund(ppn_cycle,decayed=decayed,amass_range=(50,90),ylim=[-10,-5.5])

grid(False)
legend(frameon=False)